<a href="https://colab.research.google.com/github/AARONGRANICH/Test_Data_1/blob/main/IC_MCD_Tarea_Descargando_datos_en_XML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problema Data Science

Necesito investigar sobre que novelas hicieron los poetas argentinos. Para esto crearé un dataframe con la información necesaria del archivo XML.

In [1]:
import requests
import xml.dom.minidom
import pandas as pd
import numpy as np
import xml.etree.cElementTree as et
from inspect import getmembers , isclass , isfunction
import re

In [2]:
URL = "https://raw.githubusercontent.com/mcd-unison/ing-caract/main/ejemplos/integracion/ejemplos/wikipedia-poetas.xml"

response = requests.get(URL)
with open('poetas.xml', 'wb') as file:
    file.write(response.content)

In [ ]:
dom = xml.dom.minidom.parse("./poetas.xml") 
pretty_xml_as_string = dom.toprettyxml()
print(pretty_xml_as_string)

In [4]:
# Leer XML
tree=et.parse('./poetas.xml')
root=tree.getroot()

In [5]:
# Con esto jalamos el nombre de los autores y el texto de wikipedia del respectivo autor

author_list = []
text_list = []

for autor in root.findall('{http://www.mediawiki.org/xml/export-0.10/}page'):
    
    # Nombre del autor 
    title =  autor.find('{http://www.mediawiki.org/xml/export-0.10/}title')
    # Sección de XML para buscar el texto de wikipedia
    revision =  autor.find('{http://www.mediawiki.org/xml/export-0.10/}revision')
    # Texto de wikipedia
    texto = revision.find('{http://www.mediawiki.org/xml/export-0.10/}text')
    
    author_list.append(title.text)
    text_list.append(texto.text)

In [ ]:
# Creamos un dataframe para el nombre de autores
df_author = pd.DataFrame(author_list)
df_author.columns = ['AUTHOR']

# Creamos un dataframe para el texto de wikipedia
df_text = pd.DataFrame(text_list)
df_text.columns = ['FULL_TEXT']

# Juntamos los dos dataframes en uno
final_df = pd.concat([df_author, df_text], axis=1, join='inner')
final_df

In [ ]:
# Filtramos los premios de cada autor con regex
Premios = []

for text in final_df['FULL_TEXT']:
    Premios_found = re.findall("(==\s*Premios\w*\s*==)(.*?)(\n==)", text, flags = re.DOTALL)
    Premios.append(Premios_found)
Premios

In [ ]:
# Creamos un dataframe para los ensayos

df_Premios = pd.DataFrame(Premios)
df_Premios.columns = ['Premios']
df_Premios

In [ ]:
# Actualizamos nuestro dataframe con las obras de cada autor

final_df = pd.concat([final_df, df_Premios], axis=1, join='inner')
final_df

In [ ]:
# Filtramos las obras de cada autor con regex
Obras = []

for text in final_df['FULL_TEXT']:
    Obras_found = re.findall("(==\s*Obras\w*\s*==)(.*?)(\n==)", text, flags = re.DOTALL)
    Obras.append(Obras_found)
Obras
# Creamos un dataframe para los ensayos

df_Obras = pd.DataFrame(Obras)
df_Obras.columns = ['Obras']
df_Obras
# Actualizamos nuestro dataframe con las obras de cada autor

final_df = pd.concat([final_df, df_Obras], axis=1, join='inner')
final_df

In [12]:
# Guardamos nuestro dataframe en formato parquet

final_df.to_parquet('poetas_df.parquet') 